In [ ]:
import pandas as pd
import numpy
import sklearn
import scipy
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import pdist, squareform
from math import radians, cos, sin, asin, sqrt 
from scipy.cluster.vq import whiten
from scipy.cluster.vq import *  
import matplotlib
import matplotlib.pyplot as pltpython

In [ ]:
from bokeh.io import output_notebook, show#用bokeh画图
output_notebook()

In [ ]:
import pandas as pd
import numpy as np

def coordinate(ls):# 将经纬度换算成地图上的坐标，来用bokeh画图
    k = 6378137
    ls[0] = ls[0] * (k * np.pi/180.0)
    ls[1] = np.log(np.tan((90 + ls[1]) * np.pi/360.0)) * k
    return ls

upperleft =[116.0, 40.4]
bottomright = [117.0, 39.6]

upperleft = coordinate(upperleft)
bottomright = coordinate(bottomright)
print upperleft
print bottomright

In [ ]:
rom bokeh.plotting import figure #生成北京市背景图
from bokeh.tile_providers import WMTSTileSource

China = x_range,y_range = ((upperleft[0],bottomright[0]), (bottomright[1],upperleft[1]))
fig = figure(tools='pan, wheel_zoom,hover', x_range=x_range, y_range=y_range, width = 950, height = 580)
fig.axis.visible = True
url = 'http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png'
attribution = ""
fig.add_tile(WMTSTileSource(url=url, attribution=attribution))

In [ ]:
rom bokeh.plotting import figure #生成北京市背景图
from bokeh.tile_providers import WMTSTileSource

China = x_range,y_range = ((upperleft[0],bottomright[0]), (bottomright[1],upperleft[1]))
fig = figure(tools='pan, wheel_zoom,hover', x_range=x_range, y_range=y_range, width = 950, height = 580)
fig.axis.visible = True
url = 'http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png'
attribution = ""
fig.add_tile(WMTSTileSource(url=url, attribution=attribution))

In [ ]:
fr3 = open('/data/locus/data_feature_0_1_3.csv','r') #导入用于聚类的数据，这些数据用kmeans算法聚成6类。
user_id = []
latitude = []
longitude = []
for line in fr3.readlines():
    item = [k.strip() for k in line.split(',')]
    if float(item[1])>39.820 and float(item[1])<40.001 and float(item[2])>116.238 and float(item[2])< 116.513: #判断五环数据
        user_id.append(int(item[0]))
        latitude.append(float(item[1]))
        longitude.append(float(item[2]))
lati = latitude
longi = longitude
X = pd.DataFrame ({"lat":lati,"lng":longi})
xy=numpy.array(X) #xy为用来聚类的所选用户的数据

In [ ]:
res, idx = kmeans2(xy, 6, iter=40, minit='points')  #聚类，输入：xy必须是个数组，6指的是聚类数，可改，iter是迭代次数， 输出：res 放的是经纬度，idx存的数表示原来的点在哪一类中

In [ ]:
import pandas as pd
import numpy as np
"""plot blue area"""#画出来k-means聚类效果
def wgs84_to_web_mercator(df, lon="lon", lat="lat"):# 将经纬度换算成地图上的坐标，来用bokeh画图
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df
lon = []#lon...lon5,lat...lat5分别用来记录六类的经纬度
lat = []
lon1 = []
lat1 = []
lon2 = []
lat2 = []
lon3 = []
lat3 = []
lon4 = []
lat4 = []
lon5 = []
lat5 = []
numSamples, dim = X.shape
for i in xrange(numSamples):  
    markIndex = idx[i]
    x=float(X.values[i][1])
    y=float(X.values[i][0])
    if markIndex == 0 :
        lon.append(x)
        lat.append(y)
    elif markIndex == 1 :
        lon1.append(x)
        lat1.append(y) 
    elif markIndex == 2 :
        lon2.append(x)
        lat2.append(y)
    elif markIndex == 3 :
        lon3.append(x)
        lat3.append(y)
    elif markIndex == 4 :
        lon4.append(x)
        lat4.append(y)
    else :
        lon5.append(x)
        lat5.append(y)
        
df = pd.DataFrame(dict(lon=lon,lat=lat))#将lon...lon5,lat...lat5分别转化成地图上坐标df...df5    
wgs84_to_web_mercator(df)
df1 = pd.DataFrame(dict(lon=lon1,lat=lat1))    
wgs84_to_web_mercator(df1)
df2 = pd.DataFrame(dict(lon=lon2,lat=lat2))    
wgs84_to_web_mercator(df2)
df3 = pd.DataFrame(dict(lon=lon3,lat=lat3))    
wgs84_to_web_mercator(df3)
df4 = pd.DataFrame(dict(lon=lon4,lat=lat4))    
wgs84_to_web_mercator(df4)
df5 = pd.DataFrame(dict(lon=lon5,lat=lat5))    
wgs84_to_web_mercator(df5)

In [ ]:
fig.circle(x=df['x'], y=df['y'],fill_color='blue',line_alpha=0.1, size=5)#绘制聚类图
fig.circle(x=df1['x'], y=df1['y'],fill_color='red',line_alpha=0.1, size=5)
fig.circle(x=df2['x'], y=df2['y'],fill_color='black',line_alpha=0.1, size=5)
fig.circle(x=df3['x'], y=df3['y'],fill_color='white',size=5)
fig.circle(x=df4['x'], y=df4['y'],fill_color='yellow', line_alpha=0.1,size=5)
fig.circle(x=df5['x'], y=df5['y'],fill_color='purple', line_alpha=0.1,size=5)
show(fig)

In [ ]:
numSamples, dim = X.shape#用聚类好的数据来给没有聚类好的数据进行分类
#没有分类好的数据中，如果用户的id属于已经聚类好的数据中的某一类，那么他就也属于这一类，否则属于7（哪一类都不属于），此处用字典来处理
rest_dict = {}
for i in xrange(numSamples):
    if user_id[i] not in rest_dict:
        rest_dict[user_id[i]]=[X.values[i][1],X.values[i][0],idx[i]]
other_dict = {}
fr4 = open('/data/locus/data_feature_15_16_3.csv','r')
for line in fr4.readlines():
    item = [k.strip() for k in line.split(',')]
    if int(item[0]) not in other_dict:
        other_dict[int(item[0])] =[float(item[2]),float(item[1])]

In [ ]:
for (key_1 ,value_1) in other_dict.items() :
    if key_1 in rest_dict.keys():
        value_1.append(rest_dict[key_1][2])
    else :
        value_1.append(7)

In [ ]:
import pandas as pd
import numpy as np
#画出来的是新的数据的分布
def wgs84_to_web_mercator(df, lon="lon", lat="lat"):# 将经纬度换算成地图上的坐标，来用bokeh画图
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df
other_lon = []#other_lon...other_lon5,other_lat...other_lat5分别用来记录新数据中的六类的经纬度，other_dis..other_dis5用于统计居住工作地点的距离
other_lat = []
other_lon1 = []
other_lat1 = []
other_lon2 = []
other_lat2 = []
other_lon3 = []
other_lat3 = []
other_lon4 = []
other_lat4 = []
other_lon5 = []
other_lat5 = []
other_dis=[]
other_dis1=[]
other_dis2=[]
other_dis3=[]
other_dis4=[]
other_dis5=[]
cnt= 0
for (key_1,value_1) in other_dict.items():  
    markIndex = int(other_dict[key_1][2])
    x=float(other_dict[key_1][0])
    y=float(other_dict[key_1][1])
    if markIndex == 0 :
        other_lon.append(x)
        other_lat.append(y)
    elif markIndex == 1 :
        other_lon1.append(x)
        other_lat1.append(y) 
    elif markIndex == 2 :
        other_lon2.append(x)
        other_lat2.append(y)
    elif markIndex == 3 :
        other_lon3.append(x)
        other_lat3.append(y)
    elif markIndex == 4 :
        other_lon4.append(x)
        other_lat4.append(y)
    elif markIndex == 5:
        other_lon5.append(x)
        other_lat5.append(y)
    else :
        cnt =cnt + 1
        
other_df = pd.DataFrame(dict(lon=other_lon,lat=other_lat))#将other_lon...other_lon5,other_lat...other_lat5分别转化成地图上坐标other_df...other_df5 
wgs84_to_web_mercator(other_df)
other_df1 = pd.DataFrame(dict(lon=other_lon1,lat=other_lat1))    
wgs84_to_web_mercator(other_df1)
other_df2 = pd.DataFrame(dict(lon=other_lon2,lat=other_lat2))    
wgs84_to_web_mercator(other_df2)
other_df3 = pd.DataFrame(dict(lon=other_lon3,lat=other_lat3))    
wgs84_to_web_mercator(other_df3)
other_df4 = pd.DataFrame(dict(lon=other_lon4,lat=other_lat4))    
wgs84_to_web_mercator(other_df4)
other_df5 = pd.DataFrame(dict(lon=other_lon5,lat=other_lat5))    
wgs84_to_web_mercator(other_df5)

In [ ]:
from bokeh.plotting import figure#生成新的北京市背景图
from bokeh.tile_providers import WMTSTileSource

China = x_range,y_range = ((upperleft[0],bottomright[0]), (bottomright[1],upperleft[1]))
fig1 = figure(tools='pan, wheel_zoom,hover', x_range=x_range, y_range=y_range, width = 950, height = 580)
fig1.axis.visible = True
from bokeh.plotting import figure
from bokeh.tile_providers import WMTSTileSource

China = x_range,y_range = ((upperleft[0],bottomright[0]), (bottomright[1],upperleft[1]))
fig1 = figure(tools='pan, wheel_zoom,hover', x_range=x_range, y_range=y_range, width = 950, height = 580)
fig1.axis.visible = True

In [ ]:
fig1.circle(x=df1['x'], y=df1['y'],fill_color='red', size=5)#绘制其中一个类职住的分布图
fig1.circle(x=other_df1['x'], y=other_df1['y'],fill_color='orange', size=5)
show(fig1)

In [ ]:
area_dis1=sum(other_dis)/len(other_dis)#统计平均距离
print area_dis1

In [ ]:
x = range(30)#绘制柱状图
y = [0 for i in range(30)]
for value in other_dis1:
    if value >= 30:
        value = 29.9
    y[int(value)] += 1

pltpython.bar(x,y, width = 1.0)
pltpython.show()
z=max(other_dis5)
print z